# Covid-19 Swiss model
adapted from [Tiba Razmi](https://www.linkedin.com/pulse/covid-19-swiss-model-prediction-tiba-razmi/)  
data from [Kaggle](https://www.kaggle.com/imdevskp/corona-virus-report)

## Daten importieren und vorbereiten


In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# hide warnings
import warnings
warnings.filterwarnings('ignore')

| Objekt | Bemerkung |
| :------ | :--------|
| *df*     | alle Daten aus [Kaggle](https://www.kaggle.com) |
| *df_Swiss* | eingeschränkt auf "Country/Region" == "Switzerland" |
| *predictions_model_13_03* | Spalte in df_Swiss mit polynom-fit (Datum aus Original, in wirklichkeit werden alle Daten aus *df* verwendet |
 | *temp* | Daten in **long-form** (mit Hilfe von **melt** erzeugt) |
 
 ## Daten aus Kaggle einlesen


In [44]:
df = pd.read_csv('covid_19_clean_complete.csv', delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')


df.head()

There are 19223 rows and 8 columns


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,TI,TI,0.0,0.0,3/23/20,1211.0,53.0,0.0
1,TI,TI,0.0,0.0,3/24/20,1211.0,53.0,0.0
2,Lombardia,Lombardia,0.0,0.0,3/23/20,28761.0,3776.0,0.0
3,NaN,Thailand,15.0,101.0,1/22/20,2.0,0.0,0.0
4,NaN,Japan,36.0,138.0,1/22/20,2.0,0.0,0.0


### Population by country (Daten einlesen)

In [45]:
pop=pd.read_csv('population.csv', delimiter=';')
pop_dict = dict(zip(pop["country"],pop["population"]))

## Weitere Kennzahlen berechnen und im Dataframe ergänzen
| Parameter | Bemerkung |
| :--- | :--- |
| df_selec | dataframe mit ausgewählten Ländern |
| mortality\_% | death / confirmed |
| extent | confirmed / population |
| selected_country | Einige Länder sind in Province/State unterteilt. Wenn Überseegebiete nicht zu berücksichtigen sind, z.B. Frankreich, ist <br>**df["Province/State"] == selected_country** <br>zu verwenden |

### Land wählen ('Country/Region' oder 'Province/State')
Einige Länder sind in Province/State unterteilt. Wenn Überseegebiete nicht zu berücksichtigen sind, z.B. Frankreich, ist 
**df["Province/State"]** zu verwenden

* Um zu sehen, ob ein Land in "Provinzen" unterteilt ist: Die untenstehende Zeile ausführen.

In [46]:
#df["Country/Region"].unique()
#df[df["Country/Region"].isin(["Korea, South"])]

* Jetzt hier die Länder wählen (max 6):

In [47]:
countries = ["TI","Italy","Switzerland", "Lombardia", "South Korea"]  # "Spain","Italy","Switzerland", "Germany", "South Korea"
states = ["France"]             # "France"
df_countries = df[df["Country/Region"].isin(countries)]
df_states = df[df["Province/State"].isin(states)] 
df_countries.head()
df_selec = pd.concat([df_countries, df_states], axis=0)
c_s = countries + states
c_s

['TI', 'Italy', 'Switzerland', 'Lombardia', 'South Korea', 'France']

### Kennzahlen einfügen

In [48]:
df_selec['Mortality_%'] = df_selec.Deaths / df_selec.Confirmed * 100
df_selec.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Mortality_%
0,TI,TI,0.0000,0.0000,3/23/20,1211.0,53.0,0.0,4.376548
1,TI,TI,0.0000,0.0000,3/24/20,1211.0,53.0,0.0,4.376548
2,Lombardia,Lombardia,0.0000,0.0000,3/23/20,28761.0,3776.0,0.0,13.128890
19,NaN,Italy,43.0000,12.0000,1/22/20,0.0,0.0,0.0,NaN
34,NaN,Switzerland,46.8182,8.2275,1/22/20,0.0,0.0,0.0,NaN


In [52]:
#df['Country/Region']
#pop_dict[df['Country/Region'][3]]
df_selec['Pop'] = df_selec['Country/Region']
df_selec['Pop']=df_selec['Pop'].map(pop_dict)
df_selec['Extent_%'] = df_selec.Confirmed / df_selec.Pop * 100
df_selec['Deaths%pop']=df_selec.Deaths / df_selec.Pop * 100
df_selec=df_selec.sort_values(by=['Date'])
df_selec

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Mortality_%,Pop,Extent_%,Deaths%pop
160,France,France,46.2276,2.2137,1/22/20,0.0,0.0,0.0,NaN,65273511,0.000000,0.000000
19,NaN,Italy,43.0000,12.0000,1/22/20,0.0,0.0,0.0,NaN,60461826,0.000000,0.000000
34,NaN,Switzerland,46.8182,8.2275,1/22/20,0.0,0.0,0.0,NaN,8654622,0.000000,0.000000
159,NaN,South Korea,36.0000,128.0000,1/22/20,1.0,0.0,0.0,0.000000,51269185,0.000002,0.000000
470,France,France,46.2276,2.2137,1/23/20,0.0,0.0,0.0,NaN,65273511,0.000000,0.000000
329,NaN,Italy,43.0000,12.0000,1/23/20,0.0,0.0,0.0,NaN,60461826,0.000000,0.000000
344,NaN,Switzerland,46.8182,8.2275,1/23/20,0.0,0.0,0.0,NaN,8654622,0.000000,0.000000
469,NaN,South Korea,36.0000,128.0000,1/23/20,1.0,0.0,0.0,0.000000,51269185,0.000002,0.000000
779,NaN,South Korea,36.0000,128.0000,1/24/20,2.0,0.0,0.0,0.000000,51269185,0.000004,0.000000
654,NaN,Switzerland,46.8182,8.2275,1/24/20,0.0,0.0,0.0,NaN,8654622,0.000000,0.000000


## Plotly: Mortality in % of confirmed cases

todo: 
- Kurven für ein zweites (drittes, ...) Land einfügen
- Auswahl der Länder mit widget (siehe Ref. unten)

In [50]:
# Quelle: https://www.youtube.com/watch?v=XUNaGFa9xCM
# dort gibt es ein weiteres Beispiel mit Widgets und interactive
#df.loc[df['Country/Region'] == "Italy"]

import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
py.offline.init_notebook_mode(connected=True)

layout = go.Layout(
    title='Mortality in % of confirmed cases',
    yaxis=dict(
        title='% of confirmed cases'
    ),
)

trace0 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[0],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[0], "Mortality_%"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[1],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[1],"Mortality_%"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[2],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[2], "Mortality_%"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[3],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[3],"Mortality_%"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[4],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[4], "Mortality_%"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[5],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[5],"Mortality_%"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5], layout=layout)
py.offline.iplot(fig)

## Plotly: Extent (confirmed cases in % of population)

In [12]:
# Quelle: https://www.youtube.com/watch?v=XUNaGFa9xCM
# dort gibt es ein weiteres Beispiel mit Widgets und interactive
#df.loc[df['Country/Region'] == "Italy"]

import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
py.offline.init_notebook_mode(connected=True)

layout = go.Layout(
    title='confirmed cases in % of population',
    yaxis=dict(
        title='% of population'
    ),
)

trace0 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[0],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[0], "Extent_%"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[1],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[1],"Extent_%"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[2],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[2], "Extent_%"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[3],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[3],"Extent_%"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[4],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[4], "Extent_%"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[5],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[5],"Extent_%"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5], layout=layout)
py.offline.iplot(fig)

## Plotly: Deaths in % of population 

In [13]:
# Quelle: https://www.youtube.com/watch?v=XUNaGFa9xCM
# dort gibt es ein weiteres Beispiel mit Widgets und interactive
#df.loc[df['Country/Region'] == "Italy"]

import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
py.offline.init_notebook_mode(connected=True)

layout = go.Layout(
    title='Deaths in % of population',
    yaxis=dict(
        title='% of population'
    ),
)

trace0 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[0],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[0], "Deaths%pop"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[1],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[1],"Deaths%pop"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[2],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[2], "Deaths%pop"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[3],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[3],"Deaths%pop"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[4],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[4], "Deaths%pop"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[5],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[5],"Deaths%pop"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5], layout=layout)
py.offline.iplot(fig)

In [14]:
# Quelle: https://www.youtube.com/watch?v=XUNaGFa9xCM
# dort gibt es ein weiteres Beispiel mit Widgets und interactive
#df.loc[df['Country/Region'] == "Italy"]

import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
py.offline.init_notebook_mode(connected=True)

layout = go.Layout(
    title='Deaths',
    yaxis=dict(
        title='Deaths'
    ),
)

trace0 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[0],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[0], "Deaths"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[1],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[1],"Deaths"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[2],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[2], "Deaths"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[3],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[3],"Deaths"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[4],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[4], "Deaths"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5 = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[5],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[5],"Deaths"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5], layout=layout)
py.offline.iplot(fig)